In [4]:
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

/home/asif/anaconda3/envs/medalpaca/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from peft import *

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    # config.base_model_name_or_path,
    "/home/asif/llm/model/medalpaca-7b",
    load_in_8bit=True,
    return_dict=True,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained("/home/asif/llm/model/medalpaca-7b")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.27s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Memory taken : 8GB 

In [7]:
generation_config = model.generation_config
generation_config.max_new_tokens = 400
generation_config_temperature = 0.3
generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id

In [8]:
DEVICE = "cuda"

In [9]:
def generate_response(ctx: str, options: str) -> str:
    prompt = f"""You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
{ctx}

options:
{options}

reply: 
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    # assistant_start = '<assistant>:'
    # response_start = response.find(assistant_start)
    # return response[response_start + len(assistant_start):].strip()
    return response

In [18]:
print(generate_response("The patient came to the doctor complaining of a persistent headache for the past week.","0: 'The doctor recommended a brain scan to rule out any underlying issues', 1: 'The patient was advised to take painkillers and get plenty of rest', 2: 'The doctor suggested trying acupuncture as an alternative treatment', 3: 'The patient was referred to a neurologist for further examination'"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient came to the doctor complaining of a persistent headache for the past week.

options:
0: 'The doctor recommended a brain scan to rule out any underlying issues', 1: 'The patient was advised to take painkillers and get plenty of rest', 2: 'The doctor suggested trying acupuncture as an alternative treatment', 3: 'The patient was referred to a neurologist for further examination'

reply:3


Output: 3 ===== Original: 0

In [19]:
print(generate_response("A young child has been experiencing a cough, runny nose, and sneezing for the past two days.","{0: 'The doctor prescribed a cough syrup and advised plenty of fluids', 1: 'The child should avoid exposure to cold temperatures and rest', 2: 'Nasal saline drops can help relieve congestion', 3: 'The doctor recommended a throat swab for a strep test'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A young child has been experiencing a cough, runny nose, and sneezing for the past two days.

options:
{0: 'The doctor prescribed a cough syrup and advised plenty of fluids', 1: 'The child should avoid exposure to cold temperatures and rest', 2: 'Nasal saline drops can help relieve congestion', 3: 'The doctor recommended a throat swab for a strep test'}

reply:1


Output: 1 ========= Original: 0

In [20]:
print(generate_response("A woman went to an allergist to determine the cause of her frequent allergic reactions and skin rashes.","{0: 'The allergist suggested conducting an allergy panel test', 1: 'The woman was advised to keep a diary of potential triggers', 2: 'Antihistamine medication can help alleviate symptoms', 3: 'The allergist recommended using hypoallergenic skincare products'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A woman went to an allergist to determine the cause of her frequent allergic reactions and skin rashes.

options:
{0: 'The allergist suggested conducting an allergy panel test', 1: 'The woman was advised to keep a diary of potential triggers', 2: 'Antihistamine medication can help alleviate symptoms', 3: 'The allergist recommended using hypoallergenic skincare products'}

reply:1


Output: 1 ====== Original: 1

In [21]:
print(generate_response("The patient visited the clinic for a routine check-up to monitor blood sugar levels and discuss diabetes management.","{0: 'The doctor recommended regular exercise and a balanced diet', 1: 'The patient should monitor blood sugar levels multiple times a day', 2: 'Insulin injections may be necessary to regulate glucose levels', 3: 'The doctor advised scheduling an appointment with a nutritionist'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient visited the clinic for a routine check-up to monitor blood sugar levels and discuss diabetes management.

options:
{0: 'The doctor recommended regular exercise and a balanced diet', 1: 'The patient should monitor blood sugar levels multiple times a day', 2: 'Insulin injections may be necessary to regulate glucose levels', 3: 'The doctor advised scheduling an appointment with a nutritionist'}

reply:0


Output: 0 ====== Original 3

In [22]:
print(generate_response("A man with asthma visited the pulmonologist to adjust his medication and address recent breathing difficulties.","{0: 'The pulmonologist prescribed a stronger bronchodilator for immediate relief', 1: 'The man was advised to use a peak flow meter to monitor lung function', 2: 'Inhaled corticosteroids should be taken daily for long-term control', 3: 'The doctor recommended avoiding triggers such as smoke and pet dander'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A man with asthma visited the pulmonologist to adjust his medication and address recent breathing difficulties.

options:
{0: 'The pulmonologist prescribed a stronger bronchodilator for immediate relief', 1: 'The man was advised to use a peak flow meter to monitor lung function', 2: 'Inhaled corticosteroids should be taken daily for long-term control', 3: 'The doctor recommended avoiding triggers such as smoke and pet dander'}

reply:2


Output: 2 =======  Original 2

In [23]:
print(generate_response("A teenager complained of blurry vision and difficulty reading the board at school.","{0: 'The teenager should undergo an eye exam to check for refractive errors', 1: 'Prescription glasses may be necessary to correct the vision', 2: 'Regular eye exercises can help improve focus and coordination', 3: 'The student was advised to sit closer to the front in the classroom'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A teenager complained of blurry vision and difficulty reading the board at school.

options:
{0: 'The teenager should undergo an eye exam to check for refractive errors', 1: 'Prescription glasses may be necessary to correct the vision', 2: 'Regular eye exercises can help improve focus and coordination', 3: 'The student was advised to sit closer to the front in the classroom'}

reply:1


Output: 1 ===== Original : 1

In [24]:
print(generate_response("A person sought advice from a nutritionist to develop a personalized weight loss plan.","{0: 'The nutritionist recommended a balanced diet with reduced calorie intake', 1: 'Regular physical exercise is essential for weight management', 2: 'Keeping a food journal can help track eating habits', 3: 'The person was advised to set realistic weight loss goals'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought advice from a nutritionist to develop a personalized weight loss plan.

options:
{0: 'The nutritionist recommended a balanced diet with reduced calorie intake', 1: 'Regular physical exercise is essential for weight management', 2: 'Keeping a food journal can help track eating habits', 3: 'The person was advised to set realistic weight loss goals'}

reply:0


Output: 0 ======= Original : 0

In [25]:
print(generate_response("The patient visited the dentist for a routine dental examination and cleaning.","{0: 'The dentist identified a cavity that requires filling', 1: 'Proper brushing and flossing techniques were demonstrated', 2: 'Regular dental check-ups should be scheduled every six months', 3: 'The patient was advised to use a fluoride mouthwash for added protection'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient visited the dentist for a routine dental examination and cleaning.

options:
{0: 'The dentist identified a cavity that requires filling', 1: 'Proper brushing and flossing techniques were demonstrated', 2: 'Regular dental check-ups should be scheduled every six months', 3: 'The patient was advised to use a fluoride mouthwash for added protection'}

reply:2


Output: 2 ======  Original: 2

In [26]:
print(generate_response("A person sought therapy to address anxiety and depression symptoms affecting their daily life.","{0: 'The therapist recommended cognitive-behavioral therapy sessions', 1: 'Medication options were discussed for managing symptoms', 2: 'Practicing mindfulness and relaxation techniques can help reduce anxiety', 3: 'The person was advised to reach out to a support group for additional assistance'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought therapy to address anxiety and depression symptoms affecting their daily life.

options:
{0: 'The therapist recommended cognitive-behavioral therapy sessions', 1: 'Medication options were discussed for managing symptoms', 2: 'Practicing mindfulness and relaxation techniques can help reduce anxiety', 3: 'The person was advised to reach out to a support group for additional assistance'}

reply:1


Output: 1 ========= Original : 1

In [27]:
print(generate_response("A patient visited a dermatologist with concerns about a persistent rash on their face.","{0: 'The dermatologist prescribed a topical corticosteroid for the rash', 1: 'Avoiding harsh skincare products and irritants is recommended', 2: 'Allergy testing may be necessary to identify potential triggers', 3: 'The patient was advised to protect the skin from excessive sun exposure'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient visited a dermatologist with concerns about a persistent rash on their face.

options:
{0: 'The dermatologist prescribed a topical corticosteroid for the rash', 1: 'Avoiding harsh skincare products and irritants is recommended', 2: 'Allergy testing may be necessary to identify potential triggers', 3: 'The patient was advised to protect the skin from excessive sun exposure'}

reply:1


Output: 1 ======= Original: 0

In [28]:
print(generate_response("An individual consulted a gastroenterologist regarding recurring stomach pain and digestion problems.","{0: 'The gastroenterologist recommended a comprehensive digestive system evaluation', 1: 'Dietary changes such as reducing spicy foods can alleviate symptoms', 2: 'Medication for acid reflux was prescribed', 3: 'The individual was advised to increase fiber intake for better digestion'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual consulted a gastroenterologist regarding recurring stomach pain and digestion problems.

options:
{0: 'The gastroenterologist recommended a comprehensive digestive system evaluation', 1: 'Dietary changes such as reducing spicy foods can alleviate symptoms', 2: 'Medication for acid reflux was prescribed', 3: 'The individual was advised to increase fiber intake for better digestion'}

reply:1


Output : 1 ======= Original : 2

In [29]:
print(generate_response("A person visited the orthopedic clinic with a suspected fracture in their arm after a fall.","{0: 'An X-ray was performed to confirm the fracture and determine its severity', 1: 'Immobilizing the arm with a cast is necessary for proper healing', 2: 'Pain medication was prescribed to manage discomfort', 3: 'Physical therapy sessions were recommended to regain strength and mobility'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited the orthopedic clinic with a suspected fracture in their arm after a fall.

options:
{0: 'An X-ray was performed to confirm the fracture and determine its severity', 1: 'Immobilizing the arm with a cast is necessary for proper healing', 2: 'Pain medication was prescribed to manage discomfort', 3: 'Physical therapy sessions were recommended to regain strength and mobility'}

reply:1


Output : 1 ========== Original : 1

In [30]:
print(generate_response("A patient sought help from a sleep specialist due to chronic insomnia and difficulty falling asleep.","{0: 'The sleep specialist suggested a sleep study to evaluate sleep patterns', 1: 'Sleep hygiene practices such as a consistent bedtime routine were recommended', 2: 'Prescription medication can be considered for short-term relief', 3: 'The patient was advised to limit caffeine intake and avoid electronics before bed'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient sought help from a sleep specialist due to chronic insomnia and difficulty falling asleep.

options:
{0: 'The sleep specialist suggested a sleep study to evaluate sleep patterns', 1: 'Sleep hygiene practices such as a consistent bedtime routine were recommended', 2: 'Prescription medication can be considered for short-term relief', 3: 'The patient was advised to limit caffeine intake and avoid electronics before bed'}

reply:1


Output: 1 ========== Original : 3

In [31]:
print(generate_response("A parent had questions about the recommended vaccination schedule for their newborn baby.", "{0: 'The pediatrician explained the importance of vaccines for the baby's immune system', 1: 'A vaccination chart was provided with the recommended schedule', 2: 'Possible side effects and their rarity were discussed', 3: 'The parent was advised to keep a record of the vaccinations received'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A parent had questions about the recommended vaccination schedule for their newborn baby.

options:
{0: 'The pediatrician explained the importance of vaccines for the baby's immune system', 1: 'A vaccination chart was provided with the recommended schedule', 2: 'Possible side effects and their rarity were discussed', 3: 'The parent was advised to keep a record of the vaccinations received'}

reply:1


Output : 1 ========== Original : 2

In [32]:
print(generate_response("An individual visited a cardiologist for a check-up to assess heart health and discuss preventive measures.","{0: 'The cardiologist recommended regular cardiovascular exercise', 1: 'A heart-healthy diet low in saturated fats is essential', 2: 'Monitoring blood pressure regularly can help identify potential issues', 3: 'The individual was advised to quit smoking for better heart health'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited a cardiologist for a check-up to assess heart health and discuss preventive measures.

options:
{0: 'The cardiologist recommended regular cardiovascular exercise', 1: 'A heart-healthy diet low in saturated fats is essential', 2: 'Monitoring blood pressure regularly can help identify potential issues', 3: 'The individual was advised to quit smoking for better heart health'}

reply:1


Output : 3 ========= Original : 1

# Result of medalpaca 7b 

Total examples: 15 
Correct Answers: 7
Wrong Answers : 8 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/asif/llm/evaluation/csv/hellaswag_medi.csv")

In [12]:
df["ctx_a"].iloc[16], df["cleaned_endings"].iloc[16]

('A patient visited the doctor concerned about consistently high blood pressure readings.',
 "{0: 'The doctor recommended lifestyle modifications such as a low-sodium diet', 1: 'Regular exercise can help lower blood pressure', 2: 'Prescription medication may be necessary to manage hypertension', 3: 'The patient was advised to monitor blood pressure at home regularly'}")

In [15]:
print(generate_response(df["ctx_a"].iloc[16], df["cleaned_endings"].iloc[16])), print("Original label: ", df["label"].iloc[16])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient visited the doctor concerned about consistently high blood pressure readings.

options:
{0: 'The doctor recommended lifestyle modifications such as a low-sodium diet', 1: 'Regular exercise can help lower blood pressure', 2: 'Prescription medication may be necessary to manage hypertension', 3: 'The patient was advised to monitor blood pressure at home regularly'}

reply:2
Original label:  2


(None, None)

In [16]:
print(generate_response(df["ctx_a"].iloc[17], df["cleaned_endings"].iloc[17])), print("Original label: ", df["label"].iloc[17])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual sought advice from a rheumatologist regarding persistent joint pain and stiffness.

options:
{0: 'The rheumatologist ordered blood tests to check for inflammatory markers', 1: 'Physical therapy sessions were recommended to improve joint mobility', 2: 'Medication options were discussed to manage pain and inflammation', 3: 'The individual was advised to apply warm or cold compresses for pain relief'}

reply:2
Original label:  3


(None, None)

In [17]:
print(generate_response(df["ctx_a"].iloc[18], df["cleaned_endings"].iloc[18])), print("Original label: ", df["label"].iloc[18])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person scheduled therapy sessions to address anxiety and depression symptoms affecting their daily life.

options:
{0: 'The therapist recommended cognitive-behavioral therapy to develop coping strategies', 1: 'Medication options were discussed for symptom management', 2: 'Engaging in stress-reducing activities such as meditation can be beneficial', 3: 'The person was advised to prioritize self-care and establish a support system'}

reply:1
Original label:  0


(None, None)

In [18]:
print(generate_response(df["ctx_a"].iloc[19], df["cleaned_endings"].iloc[19])), print("Original label: ", df["label"].iloc[19])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited a chiropractor with complaints of chronic back pain and limited range of motion.

options:
{0: 'The chiropractor performed spinal adjustments to alleviate pain and improve mobility', 1: 'Stretching exercises and proper posture were demonstrated', 2: 'Heat or cold therapy may provide temporary relief', 3: 'The individual was advised to avoid heavy lifting and incorporate core-strengthening exercises'}

reply:1
Original label:  1


(None, None)

In [19]:
print(generate_response(df["ctx_a"].iloc[20], df["cleaned_endings"].iloc[20])), print("Original label: ", df["label"].iloc[20])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient visited a dermatologist for a routine skin check and to assess any potential signs of skin cancer.

options:
{0: 'The dermatologist recommended a biopsy for suspicious moles', 1: 'Regular use of sunscreen and sun protection measures were emphasized', 2: 'The patient was advised to monitor any changes in the appearance of skin lesions', 3: 'Skin cancer prevention strategies such as avoiding tanning beds were discussed'}

reply:2
Original label:  0


(None, None)

In [20]:
print(generate_response(df["ctx_a"].iloc[21], df["cleaned_endings"].iloc[21])), print("Original label: ", df["label"].iloc[21])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person went to an orthopedic specialist after injuring their knee while playing sports.

options:
{0: 'The orthopedic specialist ordered an MRI to evaluate the extent of the knee injury', 1: 'Resting the knee and using ice packs can help reduce swelling', 2: 'Physical therapy sessions were recommended for rehabilitation', 3: 'Surgery may be necessary to repair the damaged knee ligaments'}

reply:2
Original label:  3


(None, None)

In [21]:
print(generate_response(df["ctx_a"].iloc[22], df["cleaned_endings"].iloc[22])), print("Original label: ", df["label"].iloc[22])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought therapy sessions to learn coping mechanisms and strategies to manage their anxiety.

options:
{0: 'The therapist recommended relaxation techniques such as deep breathing exercises', 1: 'Cognitive restructuring was discussed to challenge negative thought patterns', 2: 'Medication options were explored for anxiety symptom management', 3: 'The person was advised to maintain a consistent sleep schedule for better emotional well-being'}

reply:1
Original label:  1


(None, None)

In [22]:
print(generate_response(df["ctx_a"].iloc[23], df["cleaned_endings"].iloc[23])), print("Original label: ", df["label"].iloc[23])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An expectant mother visited an obstetrician for prenatal care, monitoring the health of both the mother and baby.

options:
{0: 'The obstetrician discussed the importance of prenatal vitamins and a balanced diet', 1: 'Regular ultrasound scans were scheduled to monitor fetal development', 2: 'The mother was advised to attend childbirth classes for preparation', 3: 'Any concerns or unusual symptoms should be promptly reported to the obstetrician'}

reply:2
Original label:  2


(None, None)

In [23]:
print(generate_response(df["ctx_a"].iloc[24], df["cleaned_endings"].iloc[24])), print("Original label: ", df["label"].iloc[24])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person experienced an allergic reaction after coming into contact with a specific food allergen.

options:
{0: 'The person was advised to carry an epinephrine auto-injector for emergency use', 1: 'Avoiding the specific allergen and reading food labels carefully is crucial', 2: 'The allergist recommended allergy testing to identify other potential triggers', 3: 'Antihistamine medication can help alleviate symptoms during a reaction'}

reply:1
Original label:  0


(None, None)

In [24]:
print(generate_response(df["ctx_a"].iloc[25], df["cleaned_endings"].iloc[25])), print("Original label: ", df["label"].iloc[25])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited a gastroenterologist with symptoms of chronic acid reflux and frequent stomach discomfort.

options:
{0: 'The gastroenterologist suggested dietary modifications such as avoiding acidic and spicy foods', 1: 'Medication for reducing stomach acid production was prescribed', 2: 'Endoscopy may be necessary to assess the esophagus and stomach', 3: 'The individual was advised to eat smaller more frequent meals to reduce reflux symptoms'}

reply:1
Original label:  2


(None, None)

In [25]:
print(generate_response(df["ctx_a"].iloc[26], df["cleaned_endings"].iloc[26])), print("Original label: ", df["label"].iloc[26])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought guidance from a pulmonologist to manage their asthma symptoms and improve lung function.

options:
{0: 'The pulmonologist prescribed an inhaler for quick-relief during asthma attacks', 1: 'Identifying and avoiding triggers such as pollen or pet dander is essential', 2: 'Regular spirometry tests were recommended to monitor lung function', 3: 'The person was advised to create an asthma action plan for better self-management'}

reply:1
Original label:  3


(None, None)

In [26]:
print(generate_response(df["ctx_a"].iloc[27], df["cleaned_endings"].iloc[27])), print("Original label: ", df["label"].iloc[27])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited an endocrinologist to discuss diabetes management and achieve better blood sugar control.

options:
{0: 'The endocrinologist recommended regular blood glucose monitoring and insulin injections', 1: 'Dietary modifications such as counting carbohydrates were discussed', 2: 'Exercise and physical activity play a crucial role in diabetes management', 3: 'The individual was advised to schedule regular follow-up appointments to track progress'}

reply:1
Original label:  0


(None, None)

In [27]:
print(generate_response(df["ctx_a"].iloc[28], df["cleaned_endings"].iloc[28])), print("Original label: ", df["label"].iloc[28])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient underwent physical therapy sessions to rehabilitate and regain strength after a sports-related injury.

options:
{0: "The physical therapist developed a customized exercise program for the patient's specific needs", 1: 'Manual therapy techniques such as massage and joint mobilization were applied', 2: 'Gradual progression of activities was recommended to avoid re-injury', 3: 'The patient was advised to perform home exercises as part of the rehabilitation process'}

reply:2
Original label:  3


(None, None)

In [28]:
print(generate_response(df["ctx_a"].iloc[29], df["cleaned_endings"].iloc[29])), print("Original label: ", df["label"].iloc[29])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited an audiologist due to concerns about hearing loss and difficulty understanding conversations.

options:
{0: 'The audiologist performed a hearing evaluation to assess the extent of hearing loss', 1: 'Hearing aids were recommended for improved auditory function', 2: 'Communication strategies such as lip-reading and using visual cues were discussed', 3: 'The individual was advised to protect the ears from loud noise to prevent further hearing damage'}

reply:2
Original label:  1


(None, None)

In [29]:
print(generate_response(df["ctx_a"].iloc[30], df["cleaned_endings"].iloc[30])), print("Original label: ", df["label"].iloc[30])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person consulted a nutritionist to develop a balanced and healthy diet plan for weight management and overall well-being.

options:
{0: 'The nutritionist emphasized the importance of consuming a variety of fruits and vegetables', 1: 'Portion control and mindful eating practices were discussed', 2: 'Including lean protein sources and whole grains in meals is recommended', 3: 'The person was advised to stay hydrated and limit intake of processed foods'}

reply:2
Original label:  2


(None, None)

In [30]:
for i in range(30,50):
    print(generate_response(df["ctx_a"].iloc[i], df["cleaned_endings"].iloc[i])), print("Original label: ", df["label"].iloc[i])

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person consulted a nutritionist to develop a balanced and healthy diet plan for weight management and overall well-being.

options:
{0: 'The nutritionist emphasized the importance of consuming a variety of fruits and vegetables', 1: 'Portion control and mindful eating practices were discussed', 2: 'Including lean protein sources and whole grains in meals is recommended', 3: 'The person was advised to stay hydrated and limit intake of processed foods'}

reply:2
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a primary care physician with symptoms of a common cold, including a runny nose and sore throat.

options:
{0: 'The primary car

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person consulted a nutritionist to develop a balanced and healthy diet plan for weight management and overall well-being.

options:
{0: 'The nutritionist emphasized the importance of consuming a variety of fruits and vegetables', 1: 'Portion control and mindful eating practices were discussed', 2: 'Including lean protein sources and whole grains in meals is recommended', 3: 'The person was advised to stay hydrated and limit intake of processed foods'}

reply:2
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a primary care physician with symptoms of a common cold, including a runny nose and sore throat.

options:
{0: 'The primary care physician recommended over-the-counter cold medications for symptom relief', 1: 'Rest and staying hydrated are crucial for recovery', 2: 'Gargling with warm saltwater can help soothe a sore throat', 3: 'The person was advised to cover the mouth and nose when coughing or sneezing'}

reply:1
Original label:  0
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient sought medical attention after experiencing a fall and suspected a possible fracture in their arm.

options:
{0: 'The healthcare provider ordered an X-ray to confirm the presence of a fracture', 1: 'Immobilization using a cast or splint may be necessary for proper healing', 2: 'Pain medication was prescribed for pain management', 3: 'Surgical intervention may be required for severe fractures'}

reply:1
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual consulted a dietitian to develop a personalized weight loss plan and adopt healthier lifestyle habits.

options:
{0: 'The dietitian recommended a combination of a balanced diet and regular physical activity', 1: 'Setting realistic goals and tracking progress can help with motivation', 2: 'Behavioral strategies such as mindful eating were discussed', 3: 'The individual was advised to seek social support and accountability for successful weight loss'}

reply:1
Original label:  1
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a neurologist after sustaining a head injury and experiencing symptoms of a concussion.

options:
{0: 'The neurologist advised rest and avoiding activities that worsen symptoms', 1: 'Gradual return to physical and cognitive activities was discussed', 2: 'Monitoring for any signs of worsening symptoms is important', 3: 'The person was advised to seek medical attention if symptoms persist or worsen'}

reply:1
Original label:  3
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual went to an optometrist to have their vision checked and discuss options for vision correction.

options:
{0: 'The optometrist prescribed corrective lenses for improved vision', 1: 'Contact lenses were recommended as an alternative to glasses', 2: 'Discussing refractive surgery options such as LASIK was part of the consultation', 3: 'The individual was advised to schedule regular eye exams for ongoing vision care'}

reply:2
Original label:  0
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient consulted an endocrinologist for evaluation and management of a suspected thyroid disorder.

options:
{0: 'The endocrinologist ordered blood tests to assess thyroid hormone levels', 1: 'Medication for thyroid hormone replacement was prescribed', 2: 'Additional imaging studies such as ultrasound may be necessary', 3: 'The patient was advised to follow up regularly for monitoring and adjustments'}

reply:2
Original label:  1
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought counseling services to address mental health concerns, including stress and anxiety.

options:
{0: 'The counselor recommended therapy sessions to explore coping mechanisms', 1: 'Developing a self-care routine and engaging in relaxation techniques can be helpful', 2: 'Medication options were discussed for symptom management', 3: 'The person was advised to reach out to a support network for emotional support'}

reply:1
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited a sleep specialist due to ongoing difficulties with sleep, such as insomnia or sleep apnea.

options:
{0: 'The sleep specialist recommended a sleep study to evaluate sleep patterns and identify any underlying disorders', 1: 'Behavioral strategies such as establishing a consistent sleep schedule were discussed', 2: 'The use of continuous positive airway pressure (CPAP) devices may be recommended for sleep apnea', 3: 'The individual was advised to limit caffeine intake and create a conducive sleep environment'}

reply:2
Original label:  3
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a cardiologist for a check-up and to manage their high blood pressure condition.

options:
{0: 'The cardiologist prescribed medication to lower blood pressure', 1: 'Lifestyle modifications such as reducing sodium intake were discussed', 2: 'Regular monitoring of blood pressure and follow-up appointments were recommended', 3: 'The person was advised to engage in regular physical activity for better cardiovascular health'}

reply:1
Original label:  0
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual consulted a rheumatologist to seek treatment options and pain management strategies for their arthritis.

options:
{0: 'The rheumatologist recommended nonsteroidal anti-inflammatory drugs (NSAIDs) for pain relief', 1: 'Physical therapy and exercises can help improve joint flexibility', 2: 'Joint injections with corticosteroids may be considered for symptom management', 3: 'The individual was advised to use assistive devices to reduce joint stress'}

reply:1
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient underwent rehabilitation therapy following a stroke to regain motor skills and improve overall functionality.

options:
{0: 'The rehabilitation therapist designed a personalized therapy plan for the patient', 1: 'Mirror therapy and electrical stimulation techniques were used during sessions', 2: 'Adaptive equipment and assistive devices can aid in daily activities', 3: 'The patient was advised to continue therapy exercises at home for better progress'}

reply:1
Original label:  1
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person underwent allergy testing to identify specific triggers that worsen their asthma symptoms.

options:
{0: 'The allergist provided a list of common asthma triggers and ways to avoid them', 1: 'Immunotherapy (allergy shots) was discussed as a potential long-term treatment', 2: 'Creating an asthma action plan based on identified triggers was recommended', 3: 'The person was advised to monitor peak flow measurements to track asthma control'}

reply:2
Original label:  0
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person attended a mental health awareness event to raise awareness about reducing stigma associated with mental health conditions.

options:
{0: 'The event emphasized the importance of open conversations about mental health', 1: 'Promoting mental health education and resources in the community was discussed', 2: 'Supporting individuals with mental health conditions through empathy and understanding was highlighted', 3: 'The person was advised to engage in advocacy efforts for mental health awareness'}

reply:2
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient underwent cataract surgery to remove a cloudy lens and improve vision clarity.

options:
{0: 'The ophthalmologist performed phacoemulsification to remove the cataract', 1: 'Intraocular lens (IOL) implantation was done for vision correction', 2: 'Postoperative care instructions such as using eye drops were provided', 3: 'The patient was advised to attend follow-up visits to monitor healing and visual acuity'}

reply:1
Original label:  1
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual experienced an allergic reaction after being exposed to a specific allergen and sought medical attention for immediate treatment.

options:
{0: 'The healthcare provider administered antihistamines and monitored vital signs', 1: 'In severe cases epinephrine injection may be necessary to counteract the allergic reaction', 2: 'Identifying and avoiding the allergen in the future was emphasized', 3: 'The person was advised to carry an epinephrine auto-injector for emergency situations'}

reply:1
Original label:  3
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient underwent a gastroenterology evaluation for chronic digestive system disorders, such as irritable bowel syndrome (IBS) or inflammatory bowel disease (IBD).

options:
{0: 'The gastroenterologist recommended dietary modifications such as a low-FODMAP diet for IBS', 1: 'Medications for symptom management and inflammation reduction were prescribed', 2: 'Additional diagnostic tests such as colonoscopy may be necessary for further evaluation', 3: 'The patient was advised to keep a food diary to identify trigger foods'}

reply:1
Original label:  2
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a dermatologist due to persistent skin rash and itching, seeking diagnosis and appropriate treatment.

options:
{0: 'The dermatologist conducted a skin examination and ordered additional tests if needed', 1: 'Prescription creams or ointments were prescribed to alleviate symptoms', 2: 'Avoiding potential irritants and maintaining good skincare practices were discussed', 3: 'The person was advised to protect the skin from excessive sun exposure'}

reply:1
Original label:  0
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited a healthcare clinic to receive routine vaccinations and ensure up-to-date immunization.

options:
{0: 'The healthcare provider administered the recommended vaccines based on age and medical history', 1: 'Educating the person about the importance of vaccinations in preventing infectious diseases was emphasized', 2: 'Potential side effects and post-vaccination care instructions were provided', 3: 'The person was advised to maintain a vaccination record for future reference'}

reply:0
Original label:  3
You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient underwent knee replacement surgery to relieve chronic pain and improve mobility in the affected joint.

options:
{0: 'The orthopedic surgeon replaced the damaged knee joint with a prosthetic implant', 1: 'Physical therapy and rehabilitation were recommended for post-surgery recovery', 2: 'Pain management strategies including medication were discussed', 3: "The patient was advised to follow the surgeon's instructions for proper wound care"}

reply:1
Original label:  1

# Results medalpaca 7b for next 30 examples
Question : 30 
Correct Answers: 11
Wrong Answers: 19

Total : 50 
Correct: 18 
Wrong : 32